In [7]:
import os, sys
import numpy as np
import json
import random

## Prepare the data --> one hot encoding matrices

In [5]:
aptamer_dataset_file = "../data/aptamer_dataset.json"
'''
Constructs a dataset that has 10,000 pairs for every class of binding affinity. 
'''
def construct_dataset():
    with open(aptamer_dataset_file, 'r') as f:
        aptamer_data = json.load(f)
    
    # Full dataset. The index of the list corresponds to the binding affinity class
    full_dataset = []
    for aptamer in aptamer_data:
        peptides = aptamer_data[aptamer]
        for p, b in peptides:
            full_dataset.append((aptamer, p))
    
    return full_dataset

In [11]:
full_dataset = construct_dataset()
random.shuffle(full_dataset)
training_set = full_dataset[:int(0.8*len(full_dataset))]
test_set = full_dataset[int(0.8*len(full_dataset)):]

In [12]:
## Takes a peptide and aptamer sequence and converts to one-hot matrix
def one_hot_peptide(sequence, seq_type='peptide'):
    if seq_type == 'peptide':
        aa_list = ['R', 'L', 'S', 'A', 'G', 'P', 'T', 'V', 'N', 'D', 'C', 'Q', 'E', 'H', 'I', 'K', 'M', 'F', 'W', 'Y']
        one_hot_peptide = np.zeros((len(sequence), len(aa_list)))
        for i in range(len(sequence)):
            aa = sequence[i]
            idx = aa_list.index(aa)
            one_hot_peptide[i][idx] = 1
        return one_hot_peptide
    else:
        na_list = ['A', 'C', 'G', 'T']
        one_hot_aptamer = np.zeros((len(sequence), len(na_list)))
        for i in range(len(sequence)):
            na = sequence[i]
            idx = na_list.index(na)
            one_hot_aptamer[i][idx] = 1
        return one_hot_aptamer
        
        

## Model --> CNN

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

In [17]:
# Define the model
class ConvNet(nn.Module):
    def __init__(self, d_value):
        super(ConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(1, 40)
        self.cnn_apt_2 = nn.Conv2d(40, 10)
        self.cnn_apt_3 = nn.Conv2d(10, 1)
        
        self.cnn_pep_1 = nn.Conv2d(1, 8)
        self.cnn_pep_2 = nn.Conv2d(8, 1)
        
        self.relu = nn.ReLU(num_parameters=1)
        
        self.sequential_pep = nn.Sequential(self.cnn_pep_1, self.relu, self.cnn_pep_2)
        self.sequential_apt = nn.Sequential(self.cnn_apt_1, self.relu, self.cnn_apt_2, self.relu, self.cnn_apt_3)
        
        self.fc1 = nn.Linear(2, 1)
       
    def forward(self, apt, pep):
        apt = apt.type(torch.FloatTensor)
        pep = pep.type(torch.FloatTensor)
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# Training Loop
import tqdm
weights = np.zeros(())
biases = np.zeros(())
for epoch in range(1):
    print("Epoch: ", epoch)
    model.train()
    # Come up with a trainloader
    for i, data in enumerate(tqdm.tqdm(trainloader)):
        # Peptide and aptamer, one-hot encode them
        pep = data['peptide']
        apt = data['aptamer']
        
        output = model(pep, apt)
        
        # Calculate g_k
        g_k = 
        
        # Update the weights
    
print('Finished Training')

## Evaluation --> compare to random